In [1]:
maria_responses = [(0.5,2), (4.5,0), (0,2), (5,0), (0.5,2), (3,0),
                   (4.5,0), (4.5,0), (0.5,2), (5,0), (5,0), (0.5,2), 
                   (1,1.5), (0.5,2), (0,2), (0,2), (3,0), (0,0.5), 
                   (4.5,0), (4.5,0), (4,0), (0,1), (0,0.5), (0,1.5),
                   (4,0), (3,0), (3,0), (0,2), (4.5,0), (4,0), (1,1), 
                   (1,1), (5,0), (4,0), (1,.5), (4,0), (5,0), (4.5,0),
                   (3,.5), (5,0), (3,0), (4,0), (2,0), (3,0), (4,0), 
                   (2.5,0), (3.5,0), (2,0), (0,1), (5,0), (3,0), (1.5,0),
                   (5,0), (0,1), (1.5,0), (5,0), (0,2), (0.5,1), (5,0),
                   (0.5,1), (0,1), (3.5,0), (0.5,1), (3.5,0), (0,1.5),
                   (0,1.5), (5,0), (1,1), (0,1), (0,1), (3,0), (5,0), 
                   (4,0), (.5,.5), (.5,.5), (0,.5), (3,0), (3.5,0),
                   (5,0), (5,0), (5,0), (4,0), (3.5,0), (4,0), (3,0), 
                   (0,2), (5,0), (2,0), (4,0), (4,0), (3,0), (4,0), (1,0),
                   (2,0), (4,0), (2,0), (2,0), (2,0), (4,0), (4,0), (3,0),
                   (4,0), (2.5,0), (.5,0), (0,.5), (3.5,0), (0,.5), (3,0),
                   (5,0), (5,0), (2,0), (5,0), (4.5,0), (2.5,0), (0,1), 
                   (0,1), (0,2), (4,0), (3.5,0), (0,2)]

In [2]:
manuel_responses = [(1,2),(4,0),(0,2),(4.5,0),(0,2),
                   (3.5,0),(5,0),(4.5,0.5),(0,2),(2,1),
                   (2,1),(0,2),(5,0),(3,1),(1,1.5),(1,1.5),
                   (3.5,0),(5,0),(0.5,2),(0.5,2),(5,0),(2,2),
                   (3.5,1.5),(2,2),(5,0),(4,.5),(4,.5),(1.5,2),
                   (4,1),(3.5,.5),(5,0),(3,.5),(5,0),(3.5,1),
                   (2,2),(5,.5),(5,0),(5,0),(4.5,.5),(5,0),(4.5,0),
                   (5,0),(4.5,0),(4.5,0),(5,0),(3.5,.5),(4,.5),
                   (3,.5),(1.5,2),(5,0),(3,.5),(2,.5),(5,0),(1,2),
                   (3,1),(5,0),(.5,2),(1.5,2),(4.5,.5),(1,2),(1,.5),
                   (4.5,.5),(1,1),(4.5,.5),(1,2),(1.5,2),(5,0),(.5,2),
                   (1,2),(1.5,2),(3.5,0),(4.5,0),(5,0),(2,1),(2,1),
                   (2,1),(5,0),(4,0),(5,0),(5,0),(5,0),(4,0),(2,.5),
                   (4.5,0),(4.5,0),(1,2),(5,0),(4,0),(5,0),(4,0),(3.5,.5),
                   (4,0),(1,2),(2,2),(5,0),(2,2),(4,0),(4,0),(5,0),(4,0),
                   (4.5,0),(5,0),(4.5,0),(3,0),(0,2),(4.5,0),(0,2),(4,0),
                   (4,0),(5,0),(5,0),(5,0),(4.5,0),(4,0),(1,1.5),(1.5,1.5),
                   (.5,2),(4,0),(4.5,0),(.5,2)]

In [3]:
print(len(maria_responses)==len(manuel_responses))

True


In [4]:
maria_accuracy_ratings = [pair[0] for pair in maria_responses]
manuel_accuracy_ratings = [pair[0] for pair in manuel_responses]
maria_sentiment_ratings = [pair[1] for pair in maria_responses]
manuel_sentiment_ratings = [pair[1] for pair in manuel_responses]

In [5]:
# Computing inter-annotator agreement using 
# Krippendorff's Alpha

#! /usr/bin/env python
# -*- coding: utf-8
'''
Python implementation of Krippendorff's alpha -- inter-rater reliability

(c)2011-17 Thomas Grill (http://grrrr.org)

Python version >= 2.4 required
'''

from __future__ import print_function
try:
    import numpy as np
except ImportError:
    np = None


def nominal_metric(a, b):
    return a != b


def interval_metric(a, b):
    return (a-b)**2


def ratio_metric(a, b):
    return ((a-b)/(a+b))**2


def krippendorff_alpha(data, metric=interval_metric, force_vecmath=False, convert_items=float, missing_items=None):
    '''
    Calculate Krippendorff's alpha (inter-rater reliability):
    
    data is in the format
    [
        {unit1:value, unit2:value, ...},  # coder 1
        {unit1:value, unit3:value, ...},   # coder 2
        ...                            # more coders
    ]
    or 
    it is a sequence of (masked) sequences (list, numpy.array, numpy.ma.array, e.g.) with rows corresponding to coders and columns to items
    
    metric: function calculating the pairwise distance
    force_vecmath: force vector math for custom metrics (numpy required)
    convert_items: function for the type conversion of items (default: float)
    missing_items: indicator for missing items (default: None)
    '''
    
    # number of coders
    m = len(data)
    
    # set of constants identifying missing values
    if missing_items is None:
        maskitems = []
    else:
        maskitems = list(missing_items)
    if np is not None:
        maskitems.append(np.ma.masked_singleton)
    
    # convert input data to a dict of items
    units = {}
    for d in data:
        try:
            # try if d behaves as a dict
            diter = d.items()
        except AttributeError:
            # sequence assumed for d
            diter = enumerate(d)
            
        for it, g in diter:
            if g not in maskitems:
                try:
                    its = units[it]
                except KeyError:
                    its = []
                    units[it] = its
                its.append(convert_items(g))


    units = dict((it, d) for it, d in units.items() if len(d) > 1)  # units with pairable values
    n = sum(len(pv) for pv in units.values())  # number of pairable values
    
    if n == 0:
        raise ValueError("No items to compare.")
    
    np_metric = (np is not None) and ((metric in (interval_metric, nominal_metric, ratio_metric)) or force_vecmath)
    
    Do = 0.
    for grades in units.values():
        if np_metric:
            gr = np.asarray(grades)
            Du = sum(np.sum(metric(gr, gri)) for gri in gr)
        else:
            Du = sum(metric(gi, gj) for gi in grades for gj in grades)
        Do += Du/float(len(grades)-1)
    Do /= float(n)

    if Do == 0:
        return 1.

    De = 0.
    for g1 in units.values():
        if np_metric:
            d1 = np.asarray(g1)
            for g2 in units.values():
                De += sum(np.sum(metric(d1, gj)) for gj in g2)
        else:
            for g2 in units.values():
                De += sum(metric(gi, gj) for gi in g1 for gj in g2)
    De /= float(n*(n-1))

    return 1.-Do/De if (Do and De) else 1. 

In [7]:
sample_accuracy_data = (
        maria_accuracy_ratings, # coder A
        manuel_accuracy_ratings, # coder B
    )
print("Krippendorff's alpha for accuracy (all): %.3f" % krippendorff_alpha(sample_accuracy_data, interval_metric))

Krippendorff's alpha for accuracy (all): 0.675


In [8]:
sample_sentiment_data = (
        maria_sentiment_ratings, # coder A
        manuel_sentiment_ratings, # coder B
    )

In [9]:
print("Krippendorff's alpha for sentiment div. (all): %.3f" % krippendorff_alpha(sample_sentiment_data, interval_metric))

Krippendorff's alpha for sentiment div. (all): 0.638


In [10]:
system_codes = [3,2,1,4,1,3,4,2,1,4,2,3,4,2,3,1,3,4,1,2,
               4,2,3,1,4,1,3,2,1,3,4,2,3,4,2,1,3,4,1,2,4,3,2,1,
               4,2,3,1,2,4,3,1,4,2,1,3,1,3,4,2,1,4,2,3,1,2,4,3,
               2,1,3,4,4,1,2,3,2,3,4,1,4,2,1,3,2,1,4,3,4,3,2,1,
               3,1,4,2,1,2,4,3,4,1,3,2,2,4,1,3,2,1,4,3,4,3,2,1,
               1,3,4,2]

In [11]:
for i in range(len(system_codes)):
    if system_codes[i]==1:
        system_codes[i] = 'Baseline'
    elif system_codes[i]==2:
        system_codes[i] = 'Modified'
    elif system_codes[i]==3:
        system_codes[i] = 'Google Translate'
    else:
        system_codes[i] = "Human"

In [12]:
# Retrieving averages by translation system

acc_human = [manuel_accuracy_ratings[i] for i in 
            range(len(manuel_accuracy_ratings)) if
            system_codes[i]=='Human'] + [maria_accuracy_ratings[i] for i in 
            range(len(maria_accuracy_ratings)) if
            system_codes[i]=='Human']
acc_bsl = [manuel_accuracy_ratings[i] for i in 
            range(len(manuel_accuracy_ratings)) if
            system_codes[i]=='Baseline'] + [maria_accuracy_ratings[i] for i in 
            range(len(maria_accuracy_ratings)) if
            system_codes[i]=='Baseline']
acc_mod = [manuel_accuracy_ratings[i] for i in 
            range(len(manuel_accuracy_ratings)) if
            system_codes[i]=='Modified'] + [maria_accuracy_ratings[i] for i in 
            range(len(maria_accuracy_ratings)) if
            system_codes[i]=='Modified']
acc_gt = [manuel_accuracy_ratings[i] for i in 
            range(len(manuel_accuracy_ratings)) if
            system_codes[i]=='Google Translate'] + [maria_accuracy_ratings[i] for i in 
            range(len(maria_accuracy_ratings)) if
            system_codes[i]=='Google Translate']

In [13]:
print(np.average(acc_human))
print(np.average(acc_bsl))
print(np.average(acc_mod))
print(np.average(acc_gt))

4.283333333333333
2.058333333333333
2.175
3.0833333333333335


In [14]:
sd_human = [manuel_sentiment_ratings[i] for i in 
            range(len(manuel_sentiment_ratings)) if
            system_codes[i]=='Human'] + [maria_sentiment_ratings[i] for i in 
            range(len(maria_sentiment_ratings)) if
            system_codes[i]=='Human']
sd_bsl = [manuel_sentiment_ratings[i] for i in 
            range(len(manuel_sentiment_ratings)) if
            system_codes[i]=='Baseline'] + [maria_sentiment_ratings[i] for i in 
            range(len(maria_sentiment_ratings)) if
            system_codes[i]=='Baseline']
sd_mod = [manuel_sentiment_ratings[i] for i in 
            range(len(manuel_sentiment_ratings)) if
            system_codes[i]=='Modified'] + [maria_sentiment_ratings[i] for i in 
            range(len(maria_sentiment_ratings)) if
            system_codes[i]=='Modified']
sd_gt = [manuel_sentiment_ratings[i] for i in 
            range(len(manuel_sentiment_ratings)) if
            system_codes[i]=='Google Translate'] + [maria_sentiment_ratings[i] for i in 
            range(len(maria_sentiment_ratings)) if
            system_codes[i]=='Google Translate']

In [15]:
print(np.average(sd_human))
print(np.average(sd_bsl))
print(np.average(sd_mod))
print(np.average(sd_gt))

0.1
0.9166666666666666
0.8083333333333333
0.425


In [16]:
idiomatic_inds = [0,1,2,3,4,5,6,7,8,9,10,11,20,21,22,23,24,25,26,27,
                  36,37,38,39,40,41,42,43,56,57,58,59,60,61,62,63,64,
                  65,66,67,72,73,74,75,92,93,94,95,112,113,114,115]
# 1, 2, 3, 6, 7, 10, 11, 15, 16, 17, 19, 24, 29

In [17]:
# Retrieving averages by translation system

idiom_acc_human = [manuel_accuracy_ratings[ind] for ind in 
            idiomatic_inds if
            system_codes[ind]=='Human'] + [maria_accuracy_ratings[ind] for ind in 
            idiomatic_inds if
            system_codes[ind]=='Human']
idiom_acc_bsl = [manuel_accuracy_ratings[ind] for ind in 
            idiomatic_inds if
            system_codes[ind]=='Baseline'] + [maria_accuracy_ratings[ind] for ind in 
            idiomatic_inds if
            system_codes[ind]=='Baseline']
idiom_acc_mod = [manuel_accuracy_ratings[ind] for ind in 
            idiomatic_inds if
            system_codes[ind]=='Modified'] + [maria_accuracy_ratings[ind] for ind in 
            idiomatic_inds if
            system_codes[ind]=='Modified']
idiom_acc_gt = [manuel_accuracy_ratings[ind] for ind in 
            idiomatic_inds if
            system_codes[ind]=='Google Translate'] + [maria_accuracy_ratings[ind] for ind in 
            idiomatic_inds if
            system_codes[ind]=='Google Translate']

In [18]:
print(np.average(idiom_acc_human))
print(np.average(idiom_acc_bsl))
print(np.average(idiom_acc_mod))
print(np.average(idiom_acc_gt))

4.4423076923076925
1.3653846153846154
2.173076923076923
2.3076923076923075


In [19]:
idiom_sd_human = [manuel_sentiment_ratings[ind] for ind in 
            idiomatic_inds if
            system_codes[ind]=='Human'] + [maria_sentiment_ratings[ind] for ind in 
            idiomatic_inds if
            system_codes[ind]=='Human']
idiom_sd_bsl = [manuel_sentiment_ratings[ind] for ind in 
            idiomatic_inds if
            system_codes[ind]=='Baseline'] + [maria_sentiment_ratings[ind] for ind in 
            idiomatic_inds if
            system_codes[ind]=='Baseline']
idiom_sd_mod = [manuel_sentiment_ratings[ind] for ind in 
            idiomatic_inds if
            system_codes[ind]=='Modified'] + [maria_sentiment_ratings[ind] for ind in 
            idiomatic_inds if
            system_codes[ind]=='Modified']
idiom_sd_gt = [manuel_sentiment_ratings[ind] for ind in 
            idiomatic_inds if
            system_codes[ind]=='Google Translate'] + [maria_sentiment_ratings[ind] for ind in 
            idiomatic_inds if
            system_codes[ind]=='Google Translate']

In [20]:
print(np.average(idiom_sd_human))
print(np.average(idiom_sd_bsl))
print(np.average(idiom_sd_mod))
print(np.average(idiom_sd_gt))

0.07692307692307693
1.2307692307692308
0.8846153846153846
0.7884615384615384


In [21]:
maria_responses_idiom = [maria_responses[ind] for ind in idiomatic_inds]
manuel_responses_idiom = [manuel_responses[ind] for ind in idiomatic_inds]

In [22]:
maria_accuracy_ratings_idiom = [pair[0] for pair in maria_responses_idiom]
manuel_accuracy_ratings_idiom = [pair[0] for pair in manuel_responses_idiom]
maria_sentiment_ratings_idiom = [pair[1] for pair in maria_responses_idiom]
manuel_sentiment_ratings_idiom = [pair[1] for pair in manuel_responses_idiom]

In [23]:
sample_acc_data_idiom = (
        maria_accuracy_ratings_idiom, # coder A
        manuel_accuracy_ratings_idiom, # coder B
    )

In [26]:
print("Krippendorff's alpha for accuracy (idiomatic): %.3f" % krippendorff_alpha(sample_acc_data_idiom, interval_metric))

Krippendorff's alpha for accuracy (idiomatic): 0.767


In [27]:
sample_sd_data_idiom = (
        maria_sentiment_ratings_idiom, # coder A
        manuel_sentiment_ratings_idiom, # coder B
    )

In [29]:
print("Krippendorff's alpha for sentiment div. (idiomatic): %.3f" % krippendorff_alpha(sample_sd_data_idiom, interval_metric))

Krippendorff's alpha for sentiment div. (idiomatic): 0.673


In [30]:
maria_qual_codes = [
                    ['IR'], [], ['MI', 'IG'], [], ['MI'], [], [], [], ['MI'],
                    [], [], ['MI'], ['IR'], ['IR'], ['MO', 'IR'], ['MO', 'IR'],
                    [], ['MO'], [], [], [], ['MO'], ['MO'], ['MO'], [], [], [], 
                    ['IR'], [], [], ['MO'], ['MO'], [], [], [], [], [], [], [], 
                    [], [], [], [], [], [], [], [], [], ['MO'], [], [], [], [],
                    ['MO'], [], [], ['O'], ['MI'], [], ['MI'], ['MO', 'IG'], [],
                    ['MO', 'IG'], [], ['MI'], ['MI'], [], ['MI', 'MO'], ['MI'],
                    ['MI'], [], [], [], [], ['MO'], ['MO'], [], [], [], [], [], 
                    [], [], [], [], ['O'], [], [], [], [], [], [], [], [], [], 
                    [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], 
                    [], [], [], [], ['IG'], ['IG'], ['O'], [], [], ['O']
                                                                               ]

In [31]:
print(len(maria_qual_codes)) # Should be 120

120


In [32]:
manuel_qual_codes = [
                     ['MO'], [], ['MI', 'MO', 'IG'], [], ['MI'], [], [], ['O'], ['MI'],
                    ['MO', 'O'], ['MO'], ['MI'], [], ['O'], ['MI', 'O'], ['MI', 'O'],
                    [], [], ['MI', 'IG'], ['MI', 'IG'], [], ['MI', 'MO', 'IG'], ['MI'],
                    ['MI', 'MO', 'IG'], [], ['MI'], ['MI'], ['MI', 'MO'], ['O'], ['MI'],
                    [], ['MO'], [], ['MO'], ['MO'], ['O'], [], [], ['O'], [], [], [], 
                    [], [], [], ['MO', 'IG'], ['MO'], ['MO', 'IG'], ['MI', 'MO', 'O'], [],
                    [], ['MI', 'O'], [], ['MI', 'MO', 'IG'], ['MI', 'MO'], [], ['O'], 
                    ['MI'], ['O'], ['MI', 'IG'], ['MO', 'IG', 'O'], ['MO'], ['MO', 'IG'],
                    ['MO'], ['MI', 'MO', 'IG'], ['MI', 'MO', 'IG'], [], ['MI', 'MO'],
                    ['MI', 'MO', 'IG'], ['MI', 'MO'], [], [], [], ['O'], ['O'], ['O'], 
                    [], [], [], [], [], [], ['O'], [], [], ['O'], [], [], [], [], [], 
                    [], ['MO', 'O'], ['MO', 'O'], [], ['MO', 'O'], [], [], [], [], [], 
                    [], [], [], ['MO', 'IG'], [], ['MO', 'IG'], [], [], [], [], [], 
                    [], [], ['MO', 'IG'], ['MO', 'IG'], ['O'], [], [], ['O']    
]

In [33]:
all_qual_codes = [l1+l2 for l1,l2 in zip(maria_qual_codes, manuel_qual_codes)]

In [34]:
print(len(manuel_qual_codes)) # Should be 120

120


In [35]:
freq_dict_hum, freq_dict_mod, freq_dict_bsl, freq_dict_gt = {},{},{},{}

In [36]:
dict_master_list = [freq_dict_hum, freq_dict_bsl, freq_dict_mod, freq_dict_gt]
assoc_codes = ['Human', 'Baseline', 'Modified', 'Google Translate']

In [37]:
for i in range(len(dict_master_list)):
    freq_dict = dict_master_list[i]
    for k in range(len(all_qual_codes)):
        if system_codes[k] == assoc_codes[i]:
            if len(all_qual_codes[k]) != 0:
                for code in all_qual_codes[k]:
                    if code not in freq_dict:
                        freq_dict[code] = 1
                    else:
                        freq_dict[code] += 1

In [38]:
print(freq_dict_hum)

{'MO': 5, 'O': 2, 'IR': 1}


In [39]:
print(freq_dict_bsl)

{'MI': 16, 'IG': 11, 'MO': 13, 'IR': 1, 'O': 15}


In [40]:
print(freq_dict_mod)

{'O': 7, 'MO': 20, 'IR': 2, 'MI': 11, 'IG': 12}


In [41]:
print(freq_dict_gt)

{'IR': 2, 'MO': 9, 'MI': 10, 'O': 3}


In [42]:
acc_ratings = maria_accuracy_ratings+manuel_accuracy_ratings

In [43]:
sd_ratings = maria_sentiment_ratings+manuel_sentiment_ratings

In [44]:
from scipy.stats import pearsonr

In [45]:
print(pearsonr(acc_ratings, sd_ratings))

(-0.7640222061855952, 3.4233813493409903e-47)


In [46]:
idiom_acc_ratings = maria_accuracy_ratings_idiom+manuel_accuracy_ratings_idiom

In [47]:
idiom_sd_ratings = maria_sentiment_ratings_idiom+manuel_sentiment_ratings_idiom

In [48]:
print(pearsonr(idiom_acc_ratings, idiom_sd_ratings))

(-0.7591136364190383, 9.900588617671247e-21)
